# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

В представленном проекте будет использована модель линейной регрессии для того, что прогнозировать данные клиентов, а также метрика r2 для того, чтобы понять, можно ли будет использовать зашифрованные данные клиентов, а также проверить, правильно ли сделан алгоритм шифрования и можно ли его использовать. Таким образом для выполнения проекта потребуется 4 библиотеки , которые представлены ниже:

In [11]:
import pandas as pd 
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

## Загрузка данных

In [2]:
df = pd.read_csv('/datasets/insurance.csv')

In [3]:
display(df)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [4]:
df.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Пропущенных значений нет, что позволяет продолжить работу

In [5]:
df = df.rename(columns={'Пол': 'sex', 
                        'Возраст': 'age', 
                        'Зарплата': 'revenue', 
                        'Члены семьи': 'family_members', 
                        'Страховые выплаты': 'insurance_payments'})

Преобразуем названия столбцов на английский язык

In [6]:
df.columns

Index(['sex', 'age', 'revenue', 'family_members', 'insurance_payments'], dtype='object')

In [20]:
corr =df.corr()
corr.style.background_gradient(cmap='coolwarm')


,sex,age,revenue,family_members,insurance_payments
sex,1.000000,0.002074,0.014910,-0.008991,0.010140
age,0.002074,1.000000,-0.019093,-0.006692,0.651030
revenue,0.014910,-0.019093,1.000000,-0.030296,-0.014963
family_members,-0.008991,-0.006692,-0.030296,1.000000,-0.036290
insurance_payments,0.010140,0.651030,-0.014963,-0.036290,1.000000


Можно заметить, что среди данных есть только два признака с сильной линейной зависимостью.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Точность линейной регрессии не изменится. 

**Обоснование:**

Используемые свойства:
$$
(AB)^T=B^T A^T
$$
$$
(AB)^{-1} = B^{-1} A^{-1}
$$
$$
A A^{-1} = A^{-1} A = E
$$
$$
AE = EA = A
$$
Доказательство:
$$
a = Xw = XEw = XPP^{-1}w = (XP)P^{-1}w = (XP)w'
$$
\
Требуется доказать, что предсказания не изменятся, имеем  $a =  Xw$,   $a' = X'w'$
\
\
$$
    w = (X^T X)^{-1} X^T y
$$
\
$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} (XP)^T y
$$
$$
w' = (P^T (X^T X) P)^{-1} P^T X^T y
$$
$$
w' = (P)^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$
$$
w' = (P)^{-1} (X^T X)^{-1} E X^T y
$$
$$
w' = (P)^{-1} (X^T X)^{-1} X^T y
$$


## Алгоритм преобразования

**Алгоритм**

Для того, чтобы зашифровать данные клиентов, можно создать случайную обратимую матрицу и ее обратную матрицу, далее умножить матрицу признаков (features) на случайную обратимую матрицу, чтобы получить зашифрованные данные. Далее, чтобы получить расшифрованные данные можно умножить зашифрованную матрицу на обратную матрицу. 

**Обоснование**

Генерация лучайной обратимой матрицы обеспечивает сохранение информации, а обратимость матрицы говорит о том, что можно будет впоследствии расшифровать данные без изменения качества модели. Умножение матрицы признаков на обратимую матрицу сохраняет корреляции, что позволит сохранить способность модели прогнозировать данные.

## Проверка алгоритма

In [7]:
features = df.drop('insurance_payments', axis=1)
target = df['insurance_payments']

In [10]:
random_matrix = np.random.rand(features.shape[1], features.shape[1]) # случ матрица
inv_rand_mat = np.linalg.inv(random_matrix) # случайная обратимая матрица

features_enc = features.dot(random_matrix) 
features_dec = features_enc.dot(inv_rand_mat)

In [17]:
model = LinearRegression()
model.fit(features, target) 
predictions = model.predict(features) 
predictions_dec = model.predict(features_dec) 

In [18]:
r2_enc = r2_score(target, predictions) 
r2_dec = r2_score(target, predictions_dec)

print('R2 для оригинальных данных:', r2_enc) 
print('-------------------------------------') 
print('R2 для преобразоввнных данных:', r2_dec)

R2 для оригинальных данных: 0.4249455028666801
-------------------------------------
R2 для преобразоввнных данных: 0.42494550286666355


## Вывод

Таким образом можно сказать, что в проекте были использована модель линейной регресии, а также два типа данных: обычные данные (без шифрования) и зашифрованные данные. Модель линейной регресии была обучена и проверены ее прогнозы на основе метрики R2 для того, чтобы понять, можно ли использовать представленный алгоритм шифрования с учетом того, что модель должна сохранять коэффициент детерминации. Можно заметить, что полученная модель и полученные данные прошли проверку, следовательно, можно использовать представленные алгоритм шифрования данных.